In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import DataPreprocessing

# Spark session

In [2]:
spark = SparkSession.builder.appName('DataPreprocessing').getOrCreate()
sc=spark.sparkContext

# Reading the data

In [3]:
file_name='Google-Playstore'

In [4]:
df = DataPreprocessing.read_data(spark)
# df.show(5)

# Get info about the data

In [5]:
# DataPreprocessing.get_info(df)

# Outliers

In [6]:
# new_df= DataPreprocessing.detect_outliers(df)


### Display boxplot for numerical columns before and after removing the outliers

In [7]:

# DataPreprocessing.boxplot_for_outliers(df, new_df)


### Safely remove the outliers

In [8]:
# print(new_df.count())

In [9]:
# from pyspark.sql.functions import col

# def remove_outliers(original_df,df_with_no_outliers):
#     '''
#     Remove the outliers from the dataset
    
#     Input:
#     df_with_no_outliers: dataframe with no outliers having only the numerical columns
#     original_df: original dataframe with all the columns

#     Output:
#     new_df: dataframe with no outliers having all the columns
#    '''
#     # take only the common rows from the two dataframes
#     new_df = original_df.join(df_with_no_outliers, on=list(original_df.columns), how='inner')
        
#     return new_df


# df= remove_outliers(df,new_df) 
# print(df.count())
# print(df.columns)

# Missing values

In [10]:
DataPreprocessing.show_nulls(df) 

                    count  percentage
Developer Website  760826   32.894268
Privacy Policy     420947   18.199619
Released            71055    3.072059
Rating              22913    0.990642
Rating Count        22894    0.989821
Minimum Android      6530    0.282324
Size                  196    0.008474
Minimum Installs      141    0.006096
Currency              135    0.005837
Installs              107    0.004626
Developer Email        38    0.001643
Maximum Installs       37    0.001600
Developer Id           33    0.001427
Price                  19    0.000821
Ad Supported            4    0.000173
In App Purchases        2    0.000086
Content Rating          0    0.000000
Editors Choice          0    0.000000
App Name                0    0.000000
Last Updated            0    0.000000
App Id                  0    0.000000
Free                    0    0.000000
Category                0    0.000000
Scraped Time            0    0.000000


## Handle missing values

### Remove columns with high percentages of nulls

In [11]:
df= DataPreprocessing.remove_useless_col(df,['Developer Website','Privacy Policy'])

### For Currency column

In [12]:
DataPreprocessing.currency_col(df)

+--------+-------+
|Currency|  count|
+--------+-------+
|     USD|2311518|
|     XXX|   1236|
|    null|    135|
|       0|     18|
|    True|     10|
|     EUR|      6|
|     INR|      5|
|     GBP|      3|
|     CAD|      2|
|     BRL|      1|
|     KRW|      1|
|     VND|      1|
|     RUB|      1|
|     TRY|      1|
|     ZAR|      1|
|     PKR|      1|
|     AUD|      1|
|    1.99|      1|
|     SGD|      1|
| 9126997|      1|
+--------+-------+



In [13]:
print("Since most of the values are USD, then we can remove this column,")
df = DataPreprocessing.remove_useless_col(df, ['Currency'])
print(f"Columns after removing useless columns:{df.columns}")

Since most of the values are USD, then we can remove this column,
Columns after removing useless columns:['App Name', 'App Id', 'Category', 'Rating', 'Rating Count', 'Installs', 'Minimum Installs', 'Maximum Installs', 'Free', 'Price', 'Size', 'Minimum Android', 'Developer Id', 'Developer Email', 'Released', 'Last Updated', 'Content Rating', 'Ad Supported', 'In App Purchases', 'Editors Choice', 'Scraped Time']


### Removes rows with nulls in Uninteresting columns

In [14]:
uninteresting_cols= ['Minimum Android','Size','Minimum Installs','Installs','Developer Email',\
                   'Developer Id','Price','Ad Supported','In App Purchases']

df=DataPreprocessing.handle_missing_values(df,cols=uninteresting_cols)

Total Number of rows : 2312944
Number of rows after dropping: 2306010


### Fill rows with nulls in interesting columns

In [15]:
interesting_num_cols=['Rating','Rating Count','Maximum Installs']
df= DataPreprocessing.handle_missing_values(df, handling_method='mean', cols=interesting_num_cols)

Total Number of rows : 2306010


In [16]:
interesting_cat_cols=['Released']
df= DataPreprocessing.handle_missing_values(df, handling_method='mode', cols=interesting_cat_cols)

Total Number of rows : 2306010


In [17]:
DataPreprocessing.show_nulls(df)

                  count  percentage
App Name              0         0.0
Minimum Android       0         0.0
Editors Choice        0         0.0
In App Purchases      0         0.0
Ad Supported          0         0.0
Content Rating        0         0.0
Last Updated          0         0.0
Released              0         0.0
Developer Email       0         0.0
Developer Id          0         0.0
Size                  0         0.0
App Id                0         0.0
Price                 0         0.0
Free                  0         0.0
Maximum Installs      0         0.0
Minimum Installs      0         0.0
Installs              0         0.0
Rating Count          0         0.0
Rating                0         0.0
Category              0         0.0
Scraped Time          0         0.0


# Explore some columns

## Size

In [18]:
DataPreprocessing.check_values_in_size_col(df)

Percentage of apps with size "Varies with device": 3.233940876232107 %


In [19]:
print('The percentage of "Varies with app" is too low, then can remove all rows with this value')

The percentage of "Varies with app" is too low, then can remove all rows with this value


In [20]:
df= DataPreprocessing.convert_size_to_bytes(df)

Converted all sizes to Bytes.


## Scraped time

In [28]:
DataPreprocessing.check_scraped_time(df)

+------------+-------+
|Scraped Time|  count|
+------------+-------+
|  2021-06-16|1794115|
|  2021-06-15| 437299|
|       False|     20|
|        True|      1|
+------------+-------+



In [30]:
print("Most of the apps were scrapped in the same 2 days, then we can remove this column.")

Most of the apps were scrapped in the same 2 days, then we can remove this column.


In [31]:
df = DataPreprocessing.remove_useless_col(df, ['Scraped time'])

In [32]:
df.columns

['App Name',
 'App Id',
 'Category',
 'Rating',
 'Rating Count',
 'Installs',
 'Minimum Installs',
 'Maximum Installs',
 'Free',
 'Price',
 'Size',
 'Minimum Android',
 'Developer Id',
 'Developer Email',
 'Released',
 'Last Updated',
 'Content Rating',
 'Ad Supported',
 'In App Purchases',
 'Editors Choice']

### Write df to csv

In [21]:
# pd_df = df.toPandas()
# pd_df.to_csv('../Dataset/'+file_name+'.csv', index=False)

### Splitting the data 

In [22]:
# DataPreprocessing.split_data()

### Prepare dataset for RDD

In [23]:
# DataPreprocessing.delimiter_to_comma()
# DataPreprocessing.delimiter_to_comma(file_name='train')
# DataPreprocessing.delimiter_to_comma(file_name='val')
# DataPreprocessing.delimiter_to_comma(file_name='test')